In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     brightness_range=[0.9,1.1], 
                                     shear_range=10)

In [4]:
x = train.drop(['id', 'digit', 'letter'], axis=1).values
x = x.reshape(-1, 28, 28, 1)
x = x/255
x_remake = []
for i in range(x.shape[0]):
    num_aug = 0
    tmp = x[i]
    tmp = tmp.reshape((1,) + tmp.shape)
    for x_aug in image_generator.flow(tmp, batch_size = 1) :
        if num_aug >= 1:
            break
        x_remake.append(x_aug[0])
        num_aug += 1
x_remake = np.array(x_remake)

x_total = np.concatenate((x, x_remake), axis=0)
print(x_total.shape)

(4096, 28, 28, 1)


In [5]:
y_data = train['digit']
y = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y[i, digit] = 1

y_remake = y.copy()
y_total = np.concatenate((y, y_remake), axis=0)
print(y_total.shape)

(4096, 10)


In [6]:
x_let = train['letter'].values
x_let = x_let[:, np.newaxis]
en = OneHotEncoder()
x_let = en.fit_transform(x_let).toarray()

x_remake_let = x_let.copy()

x_letter_total = np.concatenate((x_let, x_remake_let), axis=0)
x_letter_total.shape

(4096, 26)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True, stratify=y_total)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]
print(x_letter_train.shape)
print(x_letter_val.shape)

(3276, 28, 28, 1)
(820, 28, 28, 1)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [8]:
input1 = Input(shape=(28,28,1), name='input1')
x1 = Conv2D(64, (3,3), activation='relu', padding='same', name='conv1')(input1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2), name='pool1')(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same', name='conv2')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2), name='pool2')(x1)
x1 = Conv2D(256, (2,2), activation='relu', padding='same', name='conv3')(x1)
x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2), name='pool3')(x1)
x1 = Flatten(name='flat1')(x1)

input2 = Input(shape=(26,), name='input2')
merge = concatenate([x1, input2], name='merge')

x2 = Dense(500, activation='relu', name='hidden1')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu', name='hidden2')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu', name='hidden3')(x2)
outputs = Dense(10, activation='softmax', name='output')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input1 (InputLayer)             [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 28, 28, 64)   640         input1[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 14, 14, 64)   0           dropout[0][0]                    
______________________________________________________________________________________________

In [9]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
history = model.fit([x_train, x_letter_train], y_train, validation_data=([x_val, x_letter_val], y_val), 
                    batch_size=64, epochs=100, verbose=1, 
                    callbacks = [ModelCheckpoint('./models/{epoch:02d}-{val_accuracy:.4f}.h5',
                    monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')])

Train on 3276 samples, validate on 820 samples
Epoch 1/100
3264/3276 [============================>.] - ETA: 0s - loss: 4.6902 - accuracy: 0.1020
Epoch 00001: val_accuracy improved from -inf to 0.10488, saving model to ./models/01-0.1049.h5
3276/3276 [==============================] - 8s 3ms/sample - loss: 4.6812 - accuracy: 0.1016 - val_loss: 2.3005 - val_accuracy: 0.1049
Epoch 2/100
3264/3276 [============================>.] - ETA: 0s - loss: 2.3164 - accuracy: 0.1078
Epoch 00002: val_accuracy improved from 0.10488 to 0.11585, saving model to ./models/02-0.1159.h5
3276/3276 [==============================] - 7s 2ms/sample - loss: 2.3163 - accuracy: 0.1084 - val_loss: 2.3011 - val_accuracy: 0.1159
Epoch 3/100
3264/3276 [============================>.] - ETA: 0s - loss: 2.3065 - accuracy: 0.1131
Epoch 00003: val_accuracy improved from 0.11585 to 0.11829, saving model to ./models/03-0.1183.h5
3276/3276 [==============================] - 7s 2ms/sample - loss: 2.3065 - accuracy: 0.1136 - 

Epoch 27/100
3264/3276 [============================>.] - ETA: 0s - loss: 1.5524 - accuracy: 0.4452
Epoch 00027: val_accuracy improved from 0.39756 to 0.41707, saving model to ./models/27-0.4171.h5
3276/3276 [==============================] - 7s 2ms/sample - loss: 1.5521 - accuracy: 0.4454 - val_loss: 1.6674 - val_accuracy: 0.4171
Epoch 28/100
3264/3276 [============================>.] - ETA: 0s - loss: 1.5145 - accuracy: 0.4593
Epoch 00028: val_accuracy improved from 0.41707 to 0.43171, saving model to ./models/28-0.4317.h5
3276/3276 [==============================] - 7s 2ms/sample - loss: 1.5160 - accuracy: 0.4585 - val_loss: 1.6399 - val_accuracy: 0.4317
Epoch 29/100
3264/3276 [============================>.] - ETA: 0s - loss: 1.4633 - accuracy: 0.4847
Epoch 00029: val_accuracy did not improve from 0.43171
3276/3276 [==============================] - 7s 2ms/sample - loss: 1.4630 - accuracy: 0.4847 - val_loss: 1.6524 - val_accuracy: 0.4305
Epoch 30/100
3264/3276 [====================

Epoch 54/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.9648 - accuracy: 0.6648
Epoch 00054: val_accuracy did not improve from 0.55000
3276/3276 [==============================] - 7s 2ms/sample - loss: 0.9651 - accuracy: 0.6651 - val_loss: 1.3357 - val_accuracy: 0.5366
Epoch 55/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.9736 - accuracy: 0.6651
Epoch 00055: val_accuracy improved from 0.55000 to 0.55488, saving model to ./models/55-0.5549.h5
3276/3276 [==============================] - 7s 2ms/sample - loss: 0.9728 - accuracy: 0.6658 - val_loss: 1.3516 - val_accuracy: 0.5549
Epoch 56/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.9409 - accuracy: 0.6746
Epoch 00056: val_accuracy improved from 0.55488 to 0.55732, saving model to ./models/56-0.5573.h5
3276/3276 [==============================] - 7s 2ms/sample - loss: 0.9410 - accuracy: 0.6743 - val_loss: 1.3128 - val_accuracy: 0.5573
Epoch 57/100
3264/3276 [====================

Epoch 82/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.6090 - accuracy: 0.7929
Epoch 00082: val_accuracy did not improve from 0.59268
3276/3276 [==============================] - 7s 2ms/sample - loss: 0.6087 - accuracy: 0.7927 - val_loss: 1.3378 - val_accuracy: 0.5598
Epoch 83/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.5990 - accuracy: 0.8002
Epoch 00083: val_accuracy did not improve from 0.59268
3276/3276 [==============================] - 7s 2ms/sample - loss: 0.5997 - accuracy: 0.7998 - val_loss: 1.2866 - val_accuracy: 0.5817
Epoch 84/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.5748 - accuracy: 0.8067
Epoch 00084: val_accuracy did not improve from 0.59268
3276/3276 [==============================] - 7s 2ms/sample - loss: 0.5780 - accuracy: 0.8065 - val_loss: 1.2946 - val_accuracy: 0.5707
Epoch 85/100
3264/3276 [============================>.] - ETA: 0s - loss: 0.6898 - accuracy: 0.7754
Epoch 00085: val_accuracy did 

In [10]:
x_test = test.drop(['id', 'letter'], axis=1).values
x_test = x_test.reshape(-1, 28, 28, 1)
x_test = x_test/255
x_test.shape

(20480, 28, 28, 1)

In [11]:
#prediction = np.argmax(model.predict(x_test), axis=1)

In [12]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = np.argmax(model.predict(x_test), axis=1)

In [13]:
#submission.to_csv('data/submission1.csv', index=False)